In this notebook I have used simple Iris dataset to show how to combine different classifiers to get a more accurate classifiers.
<br>
In this example I have used two classification algorithms
* Logistic Regression
* K-nearest neighbors

Then I combined these two models to make a meta-classifier that will have more accuracy and less error rate than these two classifiers
<br>
<br>
`Note: In order to make the prediction hard for individual classifers, I have set the testing size 50%, so that it becomes more clear how ensemling can help us`

In [174]:
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder

iris = datasets.load_iris()
X, y = iris.data[50:,[1,2]], iris.target[50:]

le = LabelEncoder()
y = le.fit_transform(y)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y,random_state = 1, test_size = 0.5, stratify = y)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_std = sc.fit_transform(x_train)
x_test_std = sc.fit_transform(x_test)

All the training and test sets are now ready. Next, the models will be prepared
<br>
Note: I have used the k-fold cross validation method to check for the average accuracy of the model on 10-folds.

In [175]:
from sklearn.model_selection import cross_val_score

def clf_score(estimator):
    scores = cross_val_score(estimator=estimator,
                            X = x_train_std,
                            y = y_train,
                            cv = 10,
                            n_jobs = 1,
                            scoring = "roc_auc")
    return scores

def print_accuracy(score):
    print("Scores on 10-folds:\n {}".format(score),end = "\n\n")
    print("Average Score on 10-folds:\n {} +\- {}".format(score.mean(), score.std()))

#### Prepare Logistic Regression model

In [176]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver = "liblinear",multi_class = "auto",C = 0.001, penalty = "l2",
                       random_state = 1)
lr.fit(x_train_std,y_train)

score_lr = clf_score(lr)
print_accuracy(score_lr)

Scores on 10-folds:
 [1.         0.77777778 0.77777778 1.         0.66666667 0.5
 1.         1.         1.         1.        ]

Average Score on 10-folds:
 0.8722222222222221 +\- 0.17222222222222222


####  Prepare KNN model

In [177]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1, p = 2, metric = "minkowski")
knn.fit(x_train_std, y_train)

score_knn = clf_score(knn)
print_accuracy(score_knn)

Scores on 10-folds:
 [1.         1.         0.66666667 1.         0.66666667 0.75
 1.         0.75       1.         1.        ]

Average Score on 10-folds:
 0.8833333333333334 +\- 0.1452966314513558


#### Prepare Decision tree classifier

In [178]:
from sklearn.tree import DecisionTreeClassifier

dectree = DecisionTreeClassifier(max_depth = 1, criterion="entropy",random_state = 0)
dectree.fit(x_train_std, y_train)

score_dt = clf_score(dectree)
print_accuracy(score_dt)

Scores on 10-folds:
 [1.         1.         0.83333333 0.83333333 0.5        0.75
 1.         1.         1.         1.        ]

Average Score on 10-folds:
 0.8916666666666666 +\- 0.15833333333333333


#### Ensemble classifier

In [179]:
from sklearn.ensemble import VotingClassifier

vclf = VotingClassifier(estimators=[("lr",lr),("knn",knn),("dtree",dectree)],voting = "hard")
vclf.fit(x_train_std, y_train)

score_vclf = cross_val_score(estimator=vclf,
                            X = x_train_std,
                            y = y_train,
                            cv = 10,
                            n_jobs = 1)
print("Scores on 10-folds:\n {}".format(score_vclf),end = "\n\n")
print("Average Score on 10-folds:\n {} +\- {}".format(score_vclf.mean(), score_vclf.std()))

Scores on 10-folds:
 [1.         1.         0.83333333 1.         0.66666667 0.75
 1.         1.         1.         1.        ]

Average Score on 10-folds:
 0.925 +\- 0.12047360245667466


Error rate of each classifer

In [180]:
from sklearn.metrics import zero_one_loss, accuracy_score
def error_rate(clf):
    y_pred = clf.predict(x_test_std)
    err = 1- accuracy_score(y_test,y_pred)
    return err

error_rate(lr), error_rate(knn), error_rate(dectree)

(0.14, 0.14, 0.09999999999999998)

As we can see ensembling is pretty good when we have small dataset. However, if we reduce the test size to 10% then we can see that ensembling wouldn't make much difference.

**Thing to take awaya from here**
<br>
*Use ensembling when the dataset is small. On that dataset, train atleast 3 classifiers then combine them to make a meta-classifier of high accuracy than the individual classifier*